![logo](https://user-images.githubusercontent.com/59526258/124226124-27125b80-db3b-11eb-8ba1-488d88018ebb.png)
> **Copyright (c) 2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

# CRUD Operations Using `psycopg2`

## Introduction 
This tutorial is guide readers on how to use `psycopg2` library<sup>[[1]](#1)</sup> to perform create, read, update and delete (CRUD) operations.

## What will we accomplish?
By using `psycopy2`, readers are able to perform the following in Python:
1. Create tables and rows in `PostgreSQL`,
2. Read results of tables and rows in `PostgreSQL`,
3. Update tables and rows in `PostgreSQL`, and
4. Delete tables and rows in `PostgreSQL`.

## Instruction
You can follow along this notebook by yourself or alongside the instructor.

## Notebook Content
* [Create Operations](#create)
* [Read Operations](#read)
* [Update Operations](#update)
* [Delete Operations](#delete)
* [Exercise](#exercise)
* [References](#references)

## <a name="create">Create Operations</a>
Create operations are done using `INSERT` SQL commands. These commands are used to insert new data into an existing database.

Before we get into the actual operations, let's do some setup work first that also serves as a recap of previous notebook - `Introduction to psycopg2`.

In [ ]:
# Import package
import psycopg2
from psycopg2 import sql

### Recap

In [ ]:
# Establish connection to Postgres server
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="admin",
    database="postgres"
)

# Create a new database
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("CREATE DATABASE practice_2;")

# Disconnect from "postgres" DB
cursor.close()
conn.close()

# Establish connection to newly created "practice_2" DB
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="admin",
    database="practice_2"
)

# Create cursor object
conn.autocommit = True
cursor = conn.cursor()

Let's create a new table and populate it with some data.

In [ ]:
query = """
CREATE TABLE country
(
    country_id INTEGER,
    country_name VARCHAR(50) NOT NULL,
    country_code VARCHAR(2) UNIQUE NOT NULL,
    population INTEGER,
    yearly_change NUMERIC(5, 2),
    PRIMARY KEY (country_id)
);
"""
cursor.execute(query)

In [ ]:
# Hardcoded way of INSERT query
query = """
INSERT INTO country (country_id, country_name, country_code, population, yearly_change)
VALUES
    (1, 'Malaysia', 'MY', 32365999, 1.30),
    (2, 'Singapore', 'SG', 5850342, 0.79);
"""
cursor.execute(query)

Replace the values with the `%s` placeholders in the SQL statement string, it will automatically perform type convertion and replacement when SQL statement string & sequence of values are passed into `cursor.execute()`.

In [ ]:
# Alternative way of INSERT query by passing parameters
query = """
INSERT INTO country (country_id, country_name, country_code, population, yearly_change)
VALUES (%s, %s, %s, %s, %s);
"""
cursor.execute(query, (3, 'Indonesia', 'ID', 126476461, -0.30,))

We can validate that the table is created by querying the `pg_catalog.pg_tables` catalog. `WHERE` condition is used to filter out irrelevant system tables.

In [ ]:
query = """
SELECT pg_tables.tablename
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
schemaname != 'information_schema';
"""
cursor.execute(query)
cursor.fetchall()

##  <a name="read">Read Operations</a>

Read operations pertain with reading the inserted data. This operation corresponds with `SELECT` command in SQL. Below are some demonstrations on how to read data from tables in database.

To define SQL statement dynamically, we can use SQL string composition. <br>
Use `sql.SQL()` to create an instance of SQL string composition, then perform `format()` on it. <br>
`sql.Identifier()` usually is used for table name or column name, we cannot pass a normal string to `format()` on `SQL` string composition.

In [ ]:
# SELECT * FROM country;

SQL_query = sql.SQL("SELECT * FROM {};").format(sql.Identifier('country'))
cursor.execute(SQL_query)

cursor.fetchall()

In [ ]:
# Show what is the actual query executed in Postgresql
print(cursor.query.decode("utf-8"))

You might wonder what each column means. We can call `description` attribute for the cursor to display column name.

In [ ]:
cursor.description

Let's just display the columns `country_name` and `yearly_change` only.

In [ ]:
# SELECT country_name, population FROM country;

SQL_query = sql.SQL("""
    SELECT {},{} FROM {};
    """).format(
    sql.Identifier('country_name'),
    sql.Identifier('population'),
    sql.Identifier('country'))

cursor.execute(SQL_query)

cursor.fetchall()

In [ ]:
cursor.description

We can also perform the `JOIN` operation by passing the query in a string to `cursor.execute()`. Before that, let's quickly create another table and populate it with sample data.

In [ ]:
# create a new subcountry table
query = """
CREATE TABLE IF NOT EXISTS subcountry
(
    subcountry_id INTEGER PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
    country_id INTEGER,
    subcountry_name VARCHAR(100) NOT NULL,
    subcountry_code VARCHAR(2) NOT NULL,
    subcountry_level VARCHAR(100),
    FOREIGN KEY (country_id)
        REFERENCES country(country_id) ON DELETE CASCADE
);
"""
cursor.execute(query)

# insert some sample daa
query = """
INSERT INTO subcountry (country_id, subcountry_name, subcountry_code, subcountry_level)
VALUES
    (1, 'Johor', '01', 'State'),
    (1, 'Kedah', '02', 'State'),
    (1, 'Kelantan', '03', 'State'),
    (1, 'Wilayah Persekutuan Kuala Lumpur', '14', 'Federal Territory'),
    (3, 'Aceh', 'AC', 'Special Region'),
    (3, 'Bali', 'BA', 'Province'),
    (3, 'Bangka Belitung', 'BB', 'Province'),
    (3, 'Bengkulu', 'BE', 'Province')
RETURNING *;
"""

cursor.execute(query)

Now, let's run a `INNER JOIN` query to retrieve data from `country` and `subcountry` tables.

In [ ]:
# SELECT subcountry_name, subcountry_level, country_name
# FROM subcountry
# INNER JOIN country
# ON subcountry.country_id = country.country_id;

SQL_query = sql.SQL("""
    SELECT {}, {}, {}
    FROM {}
    INNER JOIN {}
    ON subcountry.{} = country.{};
    """).format(
    sql.Identifier('subcountry_name'),
    sql.Identifier('subcountry_level'),
    sql.Identifier('country_name'),
    sql.Identifier('subcountry'),
    sql.Identifier('country'),
    sql.Identifier('country_id'),
    sql.Identifier('country_id'))

cursor.execute(SQL_query)

cursor.fetchall()

##  <a name="update">Update Operations</a>
Update operations pertain with the act of modifying existing data. The corresponding SQL command for this type of operation is, wait for it, `UPDATE` command.

`{}` placeholders are used for table or column names formatting while `%s` placeholders are used for values formatting. <br>
We can supply index value into `{}` placeholders to indicate which sql Identifier to use.

In [ ]:
# UPDATE country
# SET population = 9999, yearly_change = 1.07
# WHERE country_id = 3
# RETURNING *;

SQL_query = sql.SQL("""
    UPDATE {0}
    SET {1} = %s, {2} = %s
    WHERE {3} = %s
    RETURNING *;
    """).format(
    sql.Identifier('country'),
    sql.Identifier('population'),
    sql.Identifier('yearly_change'),
    sql.Identifier('country_id'))

cursor.execute(SQL_query, (9999, 1.07, 3,))

cursor.fetchall()

In [ ]:
print(cursor.query.decode("utf-8"))

## <a name="delete">Delete Operations</a>

Last but not least, delete operations are used to remove existing data. The corresponding SQL command is `DELETE`.

In [ ]:
# DELETE FROM country
# WHERE country_name = 'Malaysia'
# RETURNING *;

SQL_query = sql.SQL("""
    DELETE FROM {}
    WHERE {} = %s
    RETURNING *;
    """).format(
    sql.Identifier('country'),
    sql.Identifier('country_name'))

cursor.execute(SQL_query, ('Malaysia',))

cursor.fetchall()

We can also remove the table from the database.

In [ ]:
# DROP TABLE IF EXISTS subcountry;

SQL_query = sql.SQL("DROP TABLE IF EXISTS {};").format(sql.Identifier('subcountry'))

cursor.execute(SQL_query)

In [ ]:
# DROP TABLE IF EXISTS country;

SQL_query = sql.SQL("DROP TABLE IF EXISTS {};").format(sql.Identifier('country'))

cursor.execute(SQL_query)

In [ ]:
cursor.close()
conn.close()

## <a name="exercise">Exercise</a>

Below are exercises to practice what we have just learnt. Follow along the instructions provided in the comment.

In [ ]:
# Establish connection to "practice_2" Postgres database
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="admin",
    database="practice_2"
)

# Set autocommit mode & create cursor instance
conn.autocommit = True
cursor = conn.cursor()

You are required to create another table for `titanic_train` with the columns, data types and constraints defined below.

| Column name | Data Type    | Constraint  |
| ----------- | ------------ | ----------- |
| PassengerId | INTEGER      | PRIMARY KEY |
| Survived    | BOOLEAN      |             |
| Pclass      | INTEGER      |             |
| Name        | VARCHAR(255) |             |
| Sex         | VARCHAR(255) |             |
| Age         | REAL         |             |
| SibSp       | INTEGER      |             |
| Parch       | INTEGER      |             |
| Ticket      | VARCHAR(255) |             |
| Fare        | REAL         |             |
| Cabin       | VARCHAR(255) |             |
| Embarked    | VARCHAR(255) |             |

In [ ]:
# Create titanic_train table by following the schema provided 
#    at https://www.kaggle.com/c/titanic/data?select=train.csv


In [ ]:
# Inspect created table schema
query = """
    SELECT column_name, data_type, character_maximum_length, is_nullable
    FROM INFORMATION_SCHEMA.COLUMNS 
    WHERE table_name = 'titanic_train';"""

cursor.execute(query)
cursor.fetchall()

In [ ]:
# Get valid dataset path
from pathlib import Path
dataset_path = str(Path().resolve().parent/"data/titanic_train.csv")

# Copy over data from csv file to database table
query = """
COPY titanic_train
FROM %s
DELIMITER ','
CSV HEADER;
"""
cursor.execute(query, (dataset_path,))

In [ ]:
# Read first 3 rows of records from titanic_train table
# Use "LIMIT 3;" in the end of SQL statement


In [ ]:
# Update passenger name for PassengerId 1 with your friend's name


In [ ]:
# Remove all rows where passengers did not survive


In [ ]:
# Remove titanic_train table


### Clean-up

Let's remove the database we created solely for this notebook. You may keep it but we are not going to use it for the remainder of this course.

In [ ]:
# Close cursor session & connection


In [ ]:
# Reconnect to "postgres" database


# Set autocommit mode & create cursor instance


In [ ]:
# Drop the "practice_2" database and return existing DBs


# Close current cursor session & connection


## <a name="Summary">Summary 
After this tutorial, you should have been able to:

1. Create tables and rows in `PostgreSQL`,
2. Read results of tables and rows in `PostgreSQL`,
3. Update tables and rows in `PostgreSQL`, and
4. Delete tables and rows in `PostgreSQL`.

Congratulations, that concludes this lesson.

## Contributors
**Author**<br>
[Lee Kian Yang](https://github.com/KianYang-Lee)


## <a name="references">References</a>
- <a name="1">[1]</a> [psycopg Official Website](https://www.psycopg.org/)
- [PostgreSQL Python: Create Tables](https://www.postgresqltutorial.com/postgresql-python/create-tables/)
- [Psycopy 2.9.1 Documentation: Query Parameters](https://www.psycopg.org/docs/usage.html#query-parameters)
- [Passing Table Name as a Parameter in psycopg2](https://stackoverflow.com/questions/13793399/passing-table-name-as-a-parameter-in-psycopg2)